In [15]:
!pip install transformers fschat==0.2.23 openai accelerate anthropic dotenv

In [16]:
import os
import dotenv
import wandb
hf_token = input("Enter your Hugging Face token: ").strip()
openai_key = input("Enter your OpenAI API key: ").strip()
os.environ["WANDB_MODE"] = "disabled"
os.environ["HF_TOKEN"] = hf_token
os.environ["OPENAI_API_KEY"] = openai_key

In [18]:
%cd /content/llm-adaptive-attacks
!python main.py \
  --target-model llama2-13b \
  --target-str "Sure " \
  --n-iterations 100 \
  --n-restarts 5 \
  --judge-model gpt-4-1106-preview \
  --prompt-template refined_best > attack_output.log 2>&1
!head -n 30 attack_output.log

/content/llm-adaptive-attacks
2025-07-18 05:17:28.437301: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-18 05:17:28.455159: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752815848.476483   10962 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752815848.483021   10962 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-18 05:17:28.504345: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimi

In [25]:
import json, re

with open("attack_output.log", "r") as f:
    lines = f.readlines()

results = []

for line in lines:
    line = line.encode('utf-8', 'ignore').decode()
    match = re.search(
    r"it=(\d+).*?\[best\] logprob=(-?\d+\.\d+).*?prob=([\d.]+).*?"
    r"\[curr\] logprob=(-?\d+\.\d+).*?prob=([\d.]+).*?"
    r"len_adv=(\d+)/.*?n_change=(\d+)/.*?:\s+->\s+(.*)", line)
    if match:
        results.append({
            "iteration": int(match[1]),
            "best_logprob": float(match[2]),
            "best_prob": float(match[3]),
            "curr_logprob": float(match[4]),
            "curr_prob": float(match[5]),
            "len_adv": int(match[6]),
            "n_change": int(match[7]),
            "adv_str": match[8].strip()
        })

with open("results.json", "w") as f:
    json.dump(results, f, indent=2)
raw_iters = sum(1 for l in lines if l.startswith("it="))
print(f"Raw iterations found: {raw_iters}")

print(f"Parsed {len(results)} iterations into results.json")


Raw iterations found: 39
Parsed 33 iterations into results.json


In [26]:
!head -n 300 attack_output.log
from google.colab import files
files.download("results.json")

2025-07-18 05:17:28.437301: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-18 05:17:28.455159: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752815848.476483   10962 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752815848.483021   10962 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-18 05:17:28.504345: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>